In [9]:
import os
import cv2
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import pandas as pd
import numpy as np

In [10]:
class TumorExtractor():
    
#     ("Y1.JPG",600,400)
    def __init__(self,img_file,width,height):
        self.img = cv2.resize(cv2.imread(img_file,0),(width,height))
        
    def test_img(self):
        cv2.imshow('Windows1', self.img)
        try:
            cv2.waitKey()
        except KeyboardInterrupt as e:
            print(e)
        cv2.destroyWindow('Windows1')
        
    def verify_image(self, tum):
        cv2.imshow('Windows1', tum)
        try:
            cv2.waitKey()
        except KeyboardInterrupt as e:
            print(e)
        cv2.destroyWindow('Windows1') 
        
    def extract_brain(self,ero_itr=5,dil_itr=5):
        # remove all the noise from the image
        #eg:- salt and pepper
        kernel1 = np.ones((5,5),np.float32)/25
        dst1 = cv2.filter2D(self.img,-1,kernel1)
        self.verify_image(dst1)

        #threshold the image to genrate filter
        ret2,th2 = cv2.threshold(dst1,45,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        self.verify_image(th2)
        
        #erosion of image to remove irregular shape and noise
        kernel3 = np.ones((5,5), np.uint8) 
        img_erosion = cv2.erode(th2, kernel3, iterations=ero_itr)
        self.verify_image(img_erosion)
        
        #dilation of image to maximize the region of interest
        img_dilation = cv2.dilate(img_erosion, kernel3, iterations=dil_itr)
        self.verify_image(img_dilation)
        
        contours,hierarchy = cv2.findContours(img_dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        if len(contours) != 0:
            c = max(contours, key = cv2.contourArea)
            x,y,w,h = cv2.boundingRect(c)
            new_img = self.img[y:y+h,x:x+w]
            
        return new_img
    
    def extract_tumor(self):
        #extract brain from the image
        new_img = self.extract_brain()
        self.verify_image(new_img)
        
        #apply the medium filter to smooth image and remove the noise
        median_filter = cv2.medianBlur(new_img, 5)
        self.verify_image(median_filter)
        
        #apply erosion and dilation
        kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(9,9))
        erosion = cv2.morphologyEx(median_filter, cv2.MORPH_ERODE, kernel)
        dilation = cv2.morphologyEx(erosion, cv2.MORPH_DILATE, kernel)
        
        #applying thresholding on the image to extract the tumor part from the image
        ret3,th3 = cv2.threshold(dilation,160,255,cv2.THRESH_BINARY)
        self.verify_image(th3)
        
        #apply contour filter and extract the largest contour part
        contours,hierarchy = cv2.findContours(th3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if len(contours) != 0:
            c = max(contours, key = cv2.contourArea)
            x,y,w,h = cv2.boundingRect(c)
            tumor = new_img[y:y+h,x:x+w]
            self.verify_image(tumor)
            
            new_img = cv2.rectangle(new_img, (x, y), (x + w, y + h), (255,255,255), 1)
            cv2.putText(new_img, 'Tumor Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)
            self.verify_image(new_img)
            

In [11]:
class ImageClassifier():
    
    def __init__(self,folder_list):
        self.folder_list = folder_list
        self.labels=[]
        self.image_data = []
        self.model  = SVC()
        self.stdscl = StandardScaler()
        self.x_train  = None
        self.x_test = None
        self.y_train = None
        self.y_test  = None
        
    def create_url(self):
        prediction_list  = os.listdir(self.folder_list)
        url_list = []
        for pred in prediction_list:
            url_list.append(os.path.join(os.path.abspath(self.folder_list),pred))
        dct = {}
        for url,pred in zip(url_list,prediction_list):
            images  = os.listdir(url)
            img_lst = []
            for img in images:
                img_lst.append(os.path.join(os.path.abspath(url),img))
            dct[pred]=img_lst 
#         print(dct)
        return dct

    def convert_to_standerd_image(self,img):
        try:
            
            img = cv2.imread(img,0)
            img = cv2.resize(img,(128,128))
            img = img.reshape(1,-1)
            return img
        
        except Exception as e:
            print("format incorrect ")
            return [[]]
        
    def preprocess_image(self):
        #genrate url
        img_urls  = self.create_url()
        for tags in img_urls:
            img_lst=img_urls[tags]
            for img in img_lst:
                self.image_data.append(self.convert_to_standerd_image(img)[0])
                if tags == "yes":
                    self.labels.append(1)
                else:
                    self.labels.append(0)
                    
#         print(self.image_data[0].reshape(128,128))
#         plt.imshow(self.image_data[0].reshape(128,128))

    def train_model(self):
        self.x_train,self.x_test,self.y_train,self.y_test = train_test_split(self.image_data,self.labels,test_size= 0.1,random_state=0)
        self.x_train = self.stdscl.fit_transform(self.x_train)
        self.x_test = self.stdscl.transform(self.x_test)
        self.model.fit(self.x_train,self.y_train)
    
    def predict_score(self):
        predictions = self.model.predict(self.x_test)
        print("Model Accuracy: ",accuracy_score(predictions,self.y_test))
    
    def predict_class(self,img_file):
        img = self.convert_to_standerd_image(img_file)
        img = self.stdscl.transform(img)
        pred = self.model.predict(img)
        if(pred == 1):
            te = TumorExtractor(img_file,600,400)
            te.extract_tumor()
            print("Tumor present")
        else:
            print("Tumor not present")

In [13]:
if __name__ == "__main__":
    clf = ImageClassifier("brain_tumor_dataset")
    clf.preprocess_image()
    clf.train_model()
    clf.predict_score()
    clf.predict_class("Y1.JPG")

Model Accuracy:  0.9230769230769231


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Tumor present
